In [1]:
import networkx as nx
from networkx.algorithms.community import label_propagation_communities
import networkx.algorithms.community as nx_comm
import scipy as sp
import math
from community import community_louvain

In [2]:
#Modificación de la red para que networkx no la considere como un solo nodo:
file = open("06.net", "r")
lines = []
for line in file:
    line = line.strip()
    if "Vertices" not in line and len(line.split(" ")) == 2:
        x = line.split(" ")[0]
        line = x + f' "{x}"'
    lines.append(line)
file.close()
file = open("06_mod.net", "w")
for line in lines:
    file.write(line)
    file.write("\n")
file.close()

In [3]:
G = nx.read_pajek("06_mod.net")
G = G.to_undirected()
print("Cantidad de nodos:", G.number_of_nodes())
print("Cantidad de aristas:", G.number_of_edges())

Cantidad de nodos: 1899
Cantidad de aristas: 20296


In [4]:
num_components = nx.number_connected_components(G)
print("Cantidad de componentes:",num_components)


Cantidad de componentes: 4


In [5]:
componentes = list(nx.connected_components(G))
n_total = len(G)  # Total de nodos en el grafo

# Identifica la componente gigante
componente_gigante = max(componentes, key=len)
n_gigante = len(componente_gigante)

# Calcula fracciones
fraccion_gigante = n_gigante / n_total
fraccion_pequenas = 1 - fraccion_gigante

print("Porcentaje de nodos en componente gigante:" ,fraccion_gigante*100)
print("Porcentaje de nodos en componentes pequeñas:",fraccion_pequenas )

Porcentaje de nodos en componente gigante: 99.68404423380727
Porcentaje de nodos en componentes pequeñas: 0.003159557661927326


In [6]:
# Se usa la componente gigante de acá en adelante
S = G.subgraph(max(nx.connected_components(G), key=len)).copy()
print("Cantidad de nodos:", S.number_of_nodes())
print("Cantidad de aristas:", S.number_of_edges())

Cantidad de nodos: 1893
Cantidad de aristas: 20292


In [7]:
# Si se justifica small world, para comparar con L
math.log(S.number_of_nodes())/math.log(14.617)

2.8133471368805467

In [8]:
Gsimp = nx.Graph()
for u,v,data in S.edges(data=True):
    if not Gsimp.has_edge(u,v):
        Gsimp.add_edge(u, v)

In [9]:
def max_k_core(G):
    k = 0
    while True:
        core = nx.k_core(G, k=k)
        if core.number_of_nodes() == 0:
            break
        k += 1
    return k - 1 

max_k = max_k_core(Gsimp)
print("La profundidad de k-cores es:", max_k)

La profundidad de k-cores es: 20


In [10]:
def k_shell_sizes(G):
    core_numbers = nx.core_number(G) 
    core_count = {}  
    for node, k in core_numbers.items():
        if k in core_count:
            core_count[k] += 1
        else:
            core_count[k] = 1

    sizes = {k: core_count.get(k, 0) for k in range(max_k + 1)}
    return sizes

sizes = k_shell_sizes(Gsimp)
print("Cantidad de nodos en cada k-shell:", sizes)


Cantidad de nodos en cada k-shell: {0: 0, 1: 395, 2: 228, 3: 141, 4: 118, 5: 100, 6: 75, 7: 54, 8: 64, 9: 59, 10: 44, 11: 57, 12: 39, 13: 32, 14: 63, 15: 54, 16: 29, 17: 53, 18: 55, 19: 32, 20: 201}


In [11]:
assortativity = nx.degree_assortativity_coefficient(Gsimp)
print("Coeficiente de correlación de Newman: ",assortativity)

Coeficiente de correlación de Newman:  -0.18801771829844766


In [12]:
# Obtengo su descomposición en comunidades con un algoritmo
c = list(label_propagation_communities(G))

In [13]:
# obtengo la lista de nodos de la componente conexa más grande
giant = max(nx.connected_components(G), key=len)

In [14]:
# obtengo el subgrafo consistente SOLO en la componente gigante
S = G.subgraph(max(nx.connected_components(G), key=len)).copy()

In [15]:
# A veces hay aristas repetidas y networkx lee el archivo como
# "multigrafo", para el cual no funciona k-cores. Así que genero
# acá una copia simplificada.
Gsimp = nx.Graph()
for u,v,data in G.edges(data=True):
    if not Gsimp.has_edge(u,v):
        Gsimp.add_edge(u, v)

In [16]:
clustering_individual = nx.clustering(Gsimp)
print("Coeficiente de Clustering de cada nodo:", clustering_individual)

# Promedio del Coeficiente de Clustering de la red
clustering_promedio = nx.average_clustering(Gsimp)
print(f"Coeficiente de Clustering promedio de la red: {clustering_promedio}")

# Transitividad de la red
transitividad = nx.transitivity(Gsimp)
print(f"Transitividad de la red: {transitividad}")

Coeficiente de Clustering de cada nodo: {'1': 0.09915966386554621, '2': 0.2, '3': 0.04900653843712309, '30': 0.15, '32': 0.05135781623751231, '36': 0.054136874361593465, '42': 0.03659476117103236, '44': 0.0547945205479452, '101': 0.07227891156462585, '123': 0.2, '132': 0.16774193548387098, '135': 0.08421052631578947, '146': 0.2, '159': 0, '161': 0.1282051282051282, '211': 0.11264367816091954, '255': 0, '281': 0.13732004429678848, '302': 0.06883116883116883, '312': 0.3076923076923077, '323': 0.049103808812546675, '397': 0, '477': 0.35714285714285715, '652': 0.07412587412587412, '856': 0.10909090909090909, '1014': 0.11764705882352941, '1271': 0.19047619047619047, '1312': 0.14285714285714285, '1440': 0.08658008658008658, '1626': 0.3333333333333333, '1655': 0.1619047619047619, '1675': 0.12087912087912088, '1779': 0, '1790': 0, '194': 0.057940251572327045, '313': 0.09558823529411764, '5': 0, '400': 0.00881057268722467, '1127': 0.06247086247086247, '4': 0, '9': 0.025795297372060856, '26': 0.

In [17]:
# veo el tamaño del 7-core
len(nx.k_core(Gsimp,7))

836

In [18]:
# Calculo el coeficiente de correlación de Pearson
nx.degree_pearson_correlation_coefﬁcient(G)

-0.1565234257844715

In [19]:
# Calculo el coeficiente de correlación, pero sólo para el 7-core
nx.degree_pearson_correlation_coefﬁcient(nx.k_core(Gsimp,7))

-0.16030914115197611

Modelo estructural

In [20]:
modelo = []
for i in range(10):
    modelo.append(nx.configuration_model([d for v, d in Gsimp.degree()]))

In [21]:
def simplificar_grafo(G):
    G_simple = nx.Graph()
    for u, v, data in G.edges(data=True):
        if u != v: 
            if not G_simple.has_edge(u, v):
                G_simple.add_edge(u, v)
    return G_simple

modelo_simplificado = [simplificar_grafo(graph) for graph in modelo]

In [22]:
def analizar_red(G):
    particion = community_louvain.best_partition(G)
    modularidad = community_louvain.modularity(particion, G)
    correlacion = nx.degree_pearson_correlation_coefficient(G)
    clustering = nx.average_clustering(G)
    k_cores = nx.core_number(G) 
    max_k_core = max(k_cores.values())   
    
    return modularidad, correlacion, clustering, max_k_core

# Análisis de las redes aleatorizadas
resultados_aleatorizados = [analizar_red(graph) for graph in modelo_simplificado]

print("Redes aleatorizadas:", resultados_aleatorizados)

Redes aleatorizadas: [(0.20575647545390932, -0.04771882542991358, 0.06650790811447989, 18), (0.2099279217605045, -0.04123712080841009, 0.06702921298071923, 18), (0.2053806657154684, -0.04792813938697443, 0.0705732631128785, 18), (0.20939258728339602, -0.04668922217956139, 0.07228327464007436, 19), (0.20806409650038246, -0.052787690461812316, 0.06857987626181794, 18), (0.2035905135982634, -0.044410084267806016, 0.06667280606545753, 18), (0.20395965645453001, -0.05331280515991888, 0.06941957507180926, 18), (0.20760647677730765, -0.05470264247596296, 0.06849894279880195, 18), (0.20663914262654978, -0.028824084578638654, 0.06339284816906034, 19), (0.21052417884553726, -0.043000223888562825, 0.06895481559969571, 18)]
